# DistributedJets.jl
Package that extends Jets to work with parallel distributed block operators.  This gives us a consistent way to book-keep distributed memory and computation.  It relies heavily on the community (public) DistributedArrays.jl package.

In [1]:
using Distributed

In [2]:
]add DistributedArrays DistributedJets Jets JetPack

   Updating registry at `/data/esdrd/nhwq/.julia/registries/ChevronETC`


   Updating git-repo `https://chevron.visualstudio.com/ETC-ESD-PkgRegistry.jl/_git/PkgRegistry.jl`


   Updating registry at `/data/esdrd/nhwq/.julia/registries/General`


   Updating git-repo `https://chevron.visualstudio.com/ETC-ESD-PkgRegistry.jl/_git/General.jl`


  Resolving package versions...
   Updating `/data/esdrd/nhwq/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `/data/esdrd/nhwq/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [3]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [5]:
@everywhere using DistributedArrays, DistributedJets, Jets, JetPack

We use the same blockop macro as is used in Jets, but now passing in a distributed array rather than an array

In [6]:
A = @blockop DArray(I->[JopDiagonal(rand(2)) for irow in I[1], icol in I[2]], (4,3), workers()[1:4], [2,2])

"Jet linear operator, (6,) → (8,)"

### Where are my blocks
We can use various methods to understand which processes store which blocks

In [7]:
procs(A)

2×2 Array{Int64,2}:
 2  4
 3  5

In [8]:
blockmap(A)

2×2 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:2, 1:2)  (1:2, 3:3)
 (3:4, 1:2)  (3:4, 3:3)

* pid 2 has row-blocks 1:1, and column blocks 1:2
* pid 4 has row-blocks 1:2, and column blocks 3:3
* pid 5 has row-blocks 3:4, and column blocks 1:2
* pid 6 has row-blocks 3:4, and column blocks 3:3

In [9]:
remotecall_fetch(localblockindices, 2, A)

(1:2, 1:2)

In [10]:
remotecall_fetch(localblockindices, 3, A)

(3:4, 1:2)

### Give me my blocks, please

In [11]:
getblock(A,1,1) # fetches block 1,1, and passes a copy of it from pid 2 to the master.

"Jet linear operator, (2,) → (2,)"

In [12]:
remotecall_fetch(getblock, 2, A, 1, 1) # fetch block 1,1 and pass a reference to it on pid 2

"Jet linear operator, (2,) → (2,)"

### distributed block arrays (DBArray)

In [13]:
d = rand(range(A))

8-element DBArray{Float64,Jets.BlockArray{Float64,Array{Float64,1}},Array{Jets.BlockArray{Float64,Array{Float64,1}},1}}:
 0.14248658106724488
 0.7611502980068581
 0.09661231431694439
 0.3117940791798359
 0.9796988587298028
 0.2413668217454239
 0.07458316874979354
 0.22624296141004674

In [14]:
procs(d)

2-element Array{Int64,1}:
 2
 3

In [15]:
blockmap(d)

2-element Array{UnitRange{Int64},1}:
 1:2
 3:4

In [16]:
m = rand(domain(A))

6-element DBArray{Float64,Jets.BlockArray{Float64,Array{Float64,1}},Array{Jets.BlockArray{Float64,Array{Float64,1}},1}}:
 0.23738818583217514
 0.5375588863147316
 0.8967253506997748
 0.03066171572881493
 0.6126326357294039
 0.6512382708904094

In [17]:
procs(m)

2-element Array{Int64,1}:
 2
 4

In [18]:
blockmap(m)

2-element Array{UnitRange{Int64},1}:
 1:2
 3:3

In [19]:
getblock(d, 1) # fetch block 1, and passes a copy of it from pid 2 to the master

2-element Array{Float64,1}:
 0.14248658106724488
 0.7611502980068581

In [20]:
setblock!(d, 1, ones(2)) # passes a new array from the master to pid 2, and assigns it to block 1
d

8-element DBArray{Float64,Jets.BlockArray{Float64,Array{Float64,1}},Array{Jets.BlockArray{Float64,Array{Float64,1}},1}}:
 1.0
 1.0
 0.09661231431694439
 0.3117940791798359
 0.9796988587298028
 0.2413668217454239
 0.07458316874979354
 0.22624296141004674

In [21]:
remotecall_fetch(getblock, 2, d, 1) # on pid=2 we get a reference to the block

2-element Array{Float64,1}:
 1.0
 1.0

In [22]:
@everywhere function remotegetblock_mutating(d, i)
    dᵢ = getblock(d, i)
    dᵢ .= 2.0
    nothing
end
remotecall_fetch(remotegetblock_mutating, 2, d, 1)
d

8-element DBArray{Float64,Jets.BlockArray{Float64,Array{Float64,1}},Array{Jets.BlockArray{Float64,Array{Float64,1}},1}}:
 2.0
 2.0
 0.09661231431694439
 0.3117940791798359
 0.9796988587298028
 0.2413668217454239
 0.07458316874979354
 0.22624296141004674

# Specialized distributed block operators

## tall-and-skinny
Block operators with a single column-block.  This specialization is often used in FWI.  The model is stored on the master.

In [23]:
A = @blockop DArray(I->[JopDiagonal(rand(2)) for irow=1:4, icol=1:1], (4,1))

"Jet linear operator, (2,) → (8,)"

In [24]:
blockmap(A)

4×1 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:1, 1:1)
 (2:2, 1:1)
 (3:3, 1:1)
 (4:4, 1:1)

In [25]:
d = rand(range(A))
blockmap(d)

4-element Array{UnitRange{Int64},1}:
 1:1
 2:2
 3:3
 4:4

In [26]:
m = rand(domain(A))

2-element Array{Float64,1}:
 0.5058474004789311
 0.08535928403355353

### sparse block diagonal
This is the only sparse block operator that we support.  Supporting a larger variety of sparse layouts is possible, but would require an engineering effort to build a proper sparse distributed arrays package.

In [27]:
A = @blockop DArray(
        I->[irow==icol ? JopDiagonal(rand(2)) : JopZeroBlock(JetSpace(Float64,2),JetSpace(Float64,2)) for irow in I[1], icol in I[2]],
        (4,4),
        workers()[1:4],
        [4,1]) isdiag=true

"Jet linear operator, (8,) → (8,)"

In [28]:
procs(A)

4×1 Array{Int64,2}:
 2
 3
 4
 5

In [29]:
blockmap(A)

4×1 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:1, 1:4)
 (2:2, 1:4)
 (3:3, 1:4)
 (4:4, 1:4)

In [30]:
d = rand(range(A))
blockmap(d)

4-element Array{UnitRange{Int64},1}:
 1:1
 2:2
 3:3
 4:4

In [32]:
m = rand(domain(A))
blockmap(m)

4-element Array{UnitRange{Int64},1}:
 1:1
 2:2
 3:3
 4:4